顺利通过aligo登录之后，我们看看我们可以用他实现什么操作

如果你懒得去找源码，可以直接用dir()看一下aligo的对象里有些什么：

In [ ]:
from aligo import Aligo
ali = Aligo()

list_dir = dir(ali)
print(list_dir)

这样可以大概了解一遍，在aligo的对象里都有哪些可以调用的内容。

可以看到跟分享内容相关的大多以share开头，接下来可以去看看源码如果实现这些功能，是否有我们需要的。

还有一种跟方便的方法，就是看看文档中有没有描述，或者有没有demo可以参考。

幸运的是，aligo的用例代码做的还是比较完善，我们可以直接参考使用。

我们首先注意到，把文件存储到自己网盘的功能：share_file_saveto_drive()

## 如何保存分享文件

在官方文档里，他是这样介绍的：

>保存他人分享文件
>在阿里云盘分享中, 链接末尾那一段, 代表 share_id, 即代表一个分享的唯一识别码, 例如:https://www.aliyundrive.com/s/xM6URe7xmgY
>
>```
>#其中 xM6URe7xmgY 即为 share_id
>share_id = 'xM6URe7xmgY'
># 如果一个分享是公开分享, 那么 share_pwd = '', 默认就是此值, 所以没有密码时, 直接忽略此参数即可.
># 具体情况你可以在开发工具中查看源码
>share_pwd = ''
>
># 1.如果想获取 此 share_id 对应分享信息, 可以这样做
>info = ali.get_share_info(share_id)
>
># 2.现在你想访问 此分享, 首先你需要获取 share_token
>share_token = ali.get_share_token(share_id, share_pwd)
>
># 3.现在你可以获取分享文件列表了
>share_file_list = ali.get_share_file_list(share_id, share_token.share_token)
>
># 4.这里还有一个 get_share_file 方法
>file = ali.get_share_file(share_id, file_id=share_file_list[0].file_id, share_token=share_token.share_token)
>
># 5.现在我们可以进行保存了, 比如我们保存到网盘根目录, 此时 to_parent_file_id 可以省略
>save_file = ali.share_file_saveto_drive(share_id, file_id=share_file_list[0].file_id,
>                                        share_token=share_token.share_token, 
>                                        to_parent_file_id='root')
>
>```

我们可以一步一步执行试试

In [ ]:
share_id = 'xM6URe7xmgY'
share_pwd = ''
info = ali.get_share_info(share_id)
info

可以看到，get_share_info()函数，获取到的info是一个封装了响应信息的对象，里面主要是创建者信息、共享文件图片、共享文件id、共享文件名称、更新日期等等。
这里面更新日期会比较有用，我们先记录一下。

In [ ]:
share_token = ali.get_share_token(share_id, share_pwd)
share_file_list = ali.get_share_file_list(share_id, share_token.share_token)
share_file_list

share_file_list 应该是一个list，每个元素都是BaseShareFile的对象，估计就是这个分享下的文件列表，及文件的相关信息了。由于这个分享里只有一个文件，所以list里也只有一个元素

In [ ]:
file = ali.get_share_file(share_id, file_id=share_file_list[0].file_id, share_token=share_token.share_token)
file

从`get_share_file()`函数的入参看，比较关键的是file_id，这里从list中获取第一个文件的file_id进行传入。

这里也可以大致猜到了，在aligo的体系里，每个分享的文件都是以file_id来进行唯一标记和区分的。

其中还有个parent_file_id，应该指的就是父文件夹的id。

另外还可以看到一个type，表示这个文件的类型，是folder文件夹。

但具体看，这一步获取的结果里parent_file_id这个参数和share_file_list的结果又有些不同，这个到不对我们使用造成影响，可以先不管

最后，我们试试能不能用share_file_saveto_drive()方法操作存储：

In [ ]:
save_file = ali.share_file_saveto_drive(share_id, file_id=share_file_list[0].file_id,
                                       share_token=share_token.share_token, 
                                       to_parent_file_id='root')
save_file

最后的save_file是一个保存到网盘的响应对象，我们也是主要关注里面的`file_id`，表示存储到自己的网盘空间后，这个文件的id。

然后你可以用手机刷刷自己的阿里云盘，看看是不是已经把分享的内容保存下来了。

更进一步的，每次手动截取share_id也挺麻烦，官方也给我们提供了一个截取的函数：`share_link_extract_code()`

这样又向自动化迈进了一步，如下：

In [ ]:
share_msg1 = ('「心灵奇旅 4K 原盘 REMUX」https://www.aliyundrive.com/s/FYoddEbsSwV '
              '点击链接保存，或者复制本段内容，打开「阿里云盘」APP ，无需下载极速在线查看，视频原画倍速播放')
share_msg2 = 'https://www.aliyundrive.com/s/LkUS6BrconT/folder/62074d95b39e519f60be4f73b68ac493e9977a90'

r = ali.share_link_extract_code(share_msg1)
print(r.share_id, r.share_pwd)
r = ali.share_link_extract_code(share_msg2)
print(r.share_id, r.share_pwd)

## 自己试试看吧

接下来你可以自己试试，封装一个函数，入参是阿里云盘资源分享的信息，然后打印一下这个分享文件的名称，并把里面的文件都保存到自己的网盘下，最后返回存储到个人网盘空间后的文件id。



In [ ]:
def save_share_file_to_myspace(msg):
    file_id = 0
    """
    
    """
    return file_id


#以下是阿里云盘分享资源信息
msg = '''
✥  ✥  ✥  ✥  ✥  ✥  ✥  ✥  ✥  ✥  ✥ 
✥  苹果账号、小火箭商店
✥  https://gtgmalll.com
✥  广告合作：@zaihua_bot
✥  在花频道：@TestFlightCN
✥  ✥  ✥  ✥  ✥  ✥  ✥  ✥  ✥  ✥  ✥

👇👇👇您所获取的链接👇👇👇
https://www.aliyundrive.com/s/LP9FEXbJpLs

DMCA及版权反馈、技术支持
'''

fid = save_share_file_to_myspace(msg)
print(fid)

祝你观影愉快！